In [209]:
import warnings
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import datetime

In [210]:
warnings.filterwarnings("ignore")

In [211]:
# мы сохранили наш итоговый датафрейм в файл itog_dataset.csv в файле itog.ipnyb
itog_dataset = pd.read_csv(r"C:\Users\Мой ПК\Dropbox\Мой ПК (LAPTOP-TOID7NG7)\Desktop\2 курс\AD\проект\itog_dataset.csv", sep=',')

Машинное обучение

1. Первая попытка обучения
Мы без каких-либо преобразований воспользовались "топорным" методом для загрузки в fit.

In [212]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

model1 = LinearRegression()

X = itog_dataset[['WTI', 'Brent', 'Природный газ', 'Usd_Rub', 'Количество заболевших COVID-19, чел.', 'Euro_Rub']]
y = itog_dataset['Суммарная капитализация']

# разделяем наши данные на тестовую и обучающуюся выборку
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model1.fit(X_train, y_train)
model1.coef_, model1.intercept_

(array([ 1.64358442e+02, -1.12468357e+02, -2.19876408e+02, -6.75005459e+02,
         3.67998475e-05,  5.16070570e+02]),
 22651.047543812765)

Получили следующую модель:
'Суммарная капитализация' = 22651 + 164 * 'WTI' - 112 * 'Brent' - 219 * 'Природный газ' - 675 * 'Usd_Rub' + 0,000037 * 'Количество заболевших COVID-19, чел.' + 516 * 'Euro_Rub'

In [213]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_percentage_error

model = LinearRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
y_train_pred = model.predict(X_train)

print("Test MAPE = %.4f" % mean_absolute_percentage_error(y_test, y_pred))
print("Train MAPE = %.4f" % mean_absolute_percentage_error(y_train, y_train_pred))

Test MAPE = 0.1270
Train MAPE = 0.1183


а) уменьшаем МАРЕ с помощью полиномиальной регрессии. 

In [214]:
from sklearn.preprocessing import PolynomialFeatures

polinom_transform = PolynomialFeatures(2, include_bias=False)

X_train_poly = polinom_transform.fit_transform(X_train)
X_test_poly = polinom_transform.transform(X_test)

In [215]:
model = LinearRegression()

model.fit(X_train_poly, y_train)
y_pred = model.predict(X_test_poly)
y_train_pred = model.predict(X_train_poly)

print("Test MAPE = %.4f" % mean_absolute_percentage_error(y_test, y_pred))
print("Train MAPE = %.4f" % mean_absolute_percentage_error(y_train, y_train_pred))

Test MAPE = 0.0866
Train MAPE = 0.0724


Мы добились снижения ошибки.

Используем Ridge-регрессию

In [216]:
from sklearn.linear_model import Ridge

model = Ridge(alpha=200)

model.fit(X_train_poly, y_train)
y_pred = model.predict(X_test_poly)
y_train_pred = model.predict(X_train_poly)

print("Test MAPE = %.4f" % mean_absolute_percentage_error(y_test, y_pred))
print("Train MAPE = %.4f" % mean_absolute_percentage_error(y_train, y_train_pred))

Test MAPE = 0.0863
Train MAPE = 0.0724


Ridge-регрессия не дала ощутимого результата в плане понижения ошибки MAPE.

In [217]:
# выводим основные ошибки и свойства полученной модели линейной регрессии
from sklearn.metrics import mean_squared_error, r2_score

mse = mean_squared_error(y_test, y_pred)
print("MSE:", mse)

rmse = mean_squared_error(y_test, y_pred, squared=False)
print("RMSE:", rmse)

r2 = r2_score(y_test, y_pred)
print("R-squared:", r2)

mape = mean_absolute_percentage_error(y_test, y_pred)
print("MAPE:", mape)



MSE: 5515330.743190483
RMSE: 2348.4741308327166
R-squared: 0.5941711974898123
MAPE: 0.08625688791739192


Исходя из этих данных можно сделать вывод, что наша модель не является достаточно точной для получения высококачественных прогнозов и имеет среднюю точность, так как значение MSE довольно высокое. 

Величина RMSE говорит о том, что ошибка прогноза составляет в среднем около 2348 млрд.рублей.

При этом, значение коэффициента детерминации (R-squared) находится на среднем уровне, что может говорить о том, что модель может объяснить примерно 59% изменений в зависимой переменной.

Что касается относительной ошибки (MAPE), она также не является слишком большой, показывает, что ошибки прогнозирования составляют в среднем около 8.6%.

Поэтому, может быть, стоит усовершенствовать модель для снижения значений MSE и RMSE.


б) Теперь попробуем вместо полиномиальной регрессии нормализовать данные

In [218]:
from sklearn import preprocessing

X_train_poly = preprocessing.normalize(X_train)
X_test_poly = preprocessing.normalize(X_test)

In [219]:
model = LinearRegression()

model.fit(X_train_poly, y_train)
y_pred = model.predict(X_test_poly)
y_train_pred = model.predict(X_train_poly)

print("Test MAPE = %.4f" % mean_absolute_percentage_error(y_test, y_pred))
print("Train MAPE = %.4f" % mean_absolute_percentage_error(y_train, y_train_pred))

Test MAPE = 0.1610
Train MAPE = 0.1461


не исполльзуем ridge регрессию и полиноминальную  регрессию, так как они увеличивают МАРЕ (test MAPE и train MAPE увеличвается приблизительно на 0.02.

In [220]:
mse = mean_squared_error(y_test, y_pred)
print("MSE:", mse)

rmse = mean_squared_error(y_test, y_pred, squared=False)
print("RMSE:", rmse)

r2 = r2_score(y_test, y_pred)
print("R-squared:", r2)

mape = mean_absolute_percentage_error(y_test, y_pred)
print("MAPE:", mape)

MSE: 13277818.113773178
RMSE: 3643.8740529514985
R-squared: 0.02299222368239673
MAPE: 0.16103441422615922


по сравнению с предыдущими значениями наши показатели ухудшились, вследствие чего можно сделать выывод о том, что лучше использовать полиноминальную регрессию

2) Попробуем новый тип линейной регрессии: с изначально нормализованными данными по Х

In [221]:
dfash = itog_dataset.copy()

min_value = dfash['WTI'].min()
max_value = dfash['WTI'].max()
dfash['WTI'] = (dfash['WTI'] - min_value) / (max_value - min_value)

min_value = dfash['Brent'].min()
max_value = dfash['Brent'].max()
dfash['Brent'] = (dfash['Brent'] - min_value) / (max_value - min_value)

min_value = dfash['Природный газ'].min()
max_value = dfash['Природный газ'].max()
dfash['Природный газ'] = (dfash['Природный газ'] - min_value) / (max_value - min_value)

min_value = dfash['Usd_Rub'].min()
max_value = dfash['Usd_Rub'].max()
dfash['Usd_Rub'] = (dfash['Usd_Rub'] - min_value) / (max_value - min_value)

min_value = dfash['Количество заболевших COVID-19, чел.'].min()
max_value = dfash['Количество заболевших COVID-19, чел.'].max()
dfash['Количество заболевших COVID-19, чел.'] = (dfash['Количество заболевших COVID-19, чел.'] - min_value) / (max_value - min_value)

min_value = dfash['Euro_Rub'].min()
max_value = dfash['Euro_Rub'].max()
dfash['Euro_Rub'] = (dfash['Euro_Rub'] - min_value) / (max_value - min_value)


X = dfash[['WTI', 'Brent', 'Природный газ', 'Usd_Rub', 'Количество заболевших COVID-19, чел.', 'Euro_Rub']]
y = dfash['Суммарная капитализация']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model1.fit(X_train, y_train)
model1.coef_, model1.intercept_

(array([ 26254.61759157, -11724.82621321,  -1795.29086829, -36329.46878878,
          2773.44196415,  31234.34556344]),
 6453.816469490941)

'Суммарная капитализация' = 6453 + 26254 * 'WTI' - 11724 * 'Brent' - 1795 * 'Природный газ' - 36329 * 'Usd_Rub' + 2773 * 'Количество заболевших COVID-19, чел.' + 31234 * 'Euro_Rub'

In [222]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_percentage_error

model = LinearRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
y_train_pred = model.predict(X_train)

print("Test MAPE = %.4f" % mean_absolute_percentage_error(y_test, y_pred))
print("Train MAPE = %.4f" % mean_absolute_percentage_error(y_train, y_train_pred))

Test MAPE = 0.1270
Train MAPE = 0.1183


уменьшаем МАРЕ с помощью полиномиальной регрессии

In [223]:
from sklearn.preprocessing import PolynomialFeatures

polinom_transform = PolynomialFeatures(2, include_bias=False)

X_train_poly = polinom_transform.fit_transform(X_train)
X_test_poly = polinom_transform.transform(X_test)

In [224]:
model = LinearRegression()

model.fit(X_train_poly, y_train)
y_pred = model.predict(X_test_poly)
y_train_pred = model.predict(X_train_poly)

print("Test MAPE = %.4f" % mean_absolute_percentage_error(y_test, y_pred))
print("Train MAPE = %.4f" % mean_absolute_percentage_error(y_train, y_train_pred))

Test MAPE = 0.0777
Train MAPE = 0.0667


In [225]:

mse = mean_squared_error(y_test, y_pred)
print("MSE:", mse)

rmse = mean_squared_error(y_test, y_pred, squared=False)
print("RMSE:", rmse)

r2 = r2_score(y_test, y_pred)
print("R-squared:", r2)

mape = mean_absolute_percentage_error(y_test, y_pred)
print("MAPE:", mape)

MSE: 4163379.8029844714
RMSE: 2040.4361795911361
R-squared: 0.6936503868011212
MAPE: 0.07769254216900817


По сравнению с предыдущими значениями:
1. MSE упало
2. RMSE упало
3. R квадрат возрасло
4. MAPE упало

Это позволяет нам сделать вывод о том, что данная линейная модель лучше и более точно предсказывает значение капитализации, по сравнению с предыдущими

3) новые параметры в модели (корни и натуральные логарифмы + нормализация)

берем квадратный корень от доллара и евро (их коэффициент корреляции с целевой перемнной одинаковы) и корень третьей степени от нефти (2 вида) и природного газа, так как доллар и евро схожи по характеру друг с другом (один берется с минусом, а другой - с плюсом), а природный ресурсы взаимосвязаны между собой. 

Почему именно мы взяли квадратный и кубический корни - нам так захотелось, и мы решили проверить, насколько качество модели изменится (если улучшится, то мы примем такую модель).

In [226]:
df3 = itog_dataset.copy()

df3['WTI'] = df3['WTI'] ** (1/3)
df3['Brent'] = df3['Brent'] ** (1/3)
df3['Природный газ'] = df3['Природный газ'] ** (1/3)
df3['Usd_Rub'] = df3['Usd_Rub'] ** (1/2)
df3['Euro_Rub'] = df3['Euro_Rub'] ** (1/2)

In [227]:
display(df3.isna().sum())

Дата                                    0
Суммарная капитализация                 0
WTI                                     1
Brent                                   0
Природный газ                           0
Usd_Rub                                 0
Количество заболевших COVID-19, чел.    0
Euro_Rub                                0
dtype: int64

1 пропуск: причина - отрицательные котировки на нефть! 

Поэтому сделаем нан равным 0

In [228]:
df3.fillna(0, inplace=True)

In [229]:
display(df3)

,Дата,Суммарная капитализация,WTI,Brent,Природный газ,Usd_Rub,"Количество заболевших COVID-19, чел.",Euro_Rub
0,2023-05-12,16396.946767,4.122070,4.201927,1.315399,8.827231,20718996.0,9.104268
1,2023-05-11,16294.065261,4.138289,4.216788,1.298618,8.769265,20725841.0,9.173320
2,2023-05-10,16284.351224,4.170925,4.243427,1.298815,8.721095,20729841.0,9.214516
3,2023-05-08,15763.969291,4.182390,4.254505,1.308037,8.842370,20712436.0,9.214516
4,2023-05-05,15753.358791,4.147417,4.222779,1.288056,8.796590,20690046.0,9.327304
...,...,...,...,...,...,...,...,...
1097,2018-12-07,16864.193290,3.747050,3.950857,1.649495,8.148656,0.0,8.695234
1098,2018-12-06,16668.812360,3.720269,3.916172,1.629530,8.177659,0.0,8.700868
1099,2018-12-05,16954.240210,3.753685,3.948507,1.647164,8.158854,0.0,8.698057
1100,2018-12-04,16881.495090,3.762183,3.959593,1.645688,8.175579,0.0,8.682592


Делаем нормализацию

In [230]:
min_value = df3['WTI'].min()
max_value = df3['WTI'].max()
df3['WTI'] = (df3['WTI'] - min_value) / (max_value - min_value)

min_value = df3['Brent'].min()
max_value = df3['Brent'].max()
df3['Brent'] = (df3['Brent'] - min_value) / (max_value - min_value)

min_value = df3['Природный газ'].min()
max_value = df3['Природный газ'].max()
df3['Природный газ'] = (df3['Природный газ'] - min_value) / (max_value - min_value)

min_value = df3['Usd_Rub'].min()
max_value = df3['Usd_Rub'].max()
df3['Usd_Rub'] = (df3['Usd_Rub'] - min_value) / (max_value - min_value)

min_value = df3['Количество заболевших COVID-19, чел.'].min()
max_value = df3['Количество заболевших COVID-19, чел.'].max()
df3['Количество заболевших COVID-19, чел.'] = (df3['Количество заболевших COVID-19, чел.'] - min_value) / (max_value - min_value)

min_value = df3['Euro_Rub'].min()
max_value = df3['Euro_Rub'].max()
df3['Euro_Rub'] = (df3['Euro_Rub'] - min_value) / (max_value - min_value)

In [231]:
display(df3)

,Дата,Суммарная капитализация,WTI,Brent,Природный газ,Usd_Rub,"Количество заболевших COVID-19, чел.",Euro_Rub
0,2023-05-12,16396.946767,0.830867,0.660867,0.177292,0.535861,0.274913,0.544969
1,2023-05-11,16294.065261,0.834136,0.667336,0.160318,0.517086,0.275004,0.565396
2,2023-05-10,16284.351224,0.840715,0.678932,0.160518,0.501483,0.275057,0.577583
3,2023-05-08,15763.969291,0.843026,0.683755,0.169846,0.540765,0.274826,0.577583
4,2023-05-05,15753.358791,0.835976,0.669944,0.149636,0.525937,0.274529,0.610949
...,...,...,...,...,...,...,...,...
1097,2018-12-07,16864.193290,0.755276,0.551575,0.515219,0.316066,0.000000,0.423965
1098,2018-12-06,16668.812360,0.749878,0.536476,0.495025,0.325460,0.000000,0.425632
1099,2018-12-05,16954.240210,0.756614,0.550551,0.512861,0.319369,0.000000,0.424800
1100,2018-12-04,16881.495090,0.758326,0.555378,0.511369,0.324787,0.000000,0.420225


In [232]:
df3.isna().sum()

Дата                                    0
Суммарная капитализация                 0
WTI                                     0
Brent                                   0
Природный газ                           0
Usd_Rub                                 0
Количество заболевших COVID-19, чел.    0
Euro_Rub                                0
dtype: int64

In [233]:
X = df3[['WTI', 'Brent', 'Природный газ', 'Usd_Rub', 'Количество заболевших COVID-19, чел.', 'Euro_Rub']]
y = df3['Суммарная капитализация']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model1.fit(X_train, y_train)
model1.coef_, model1.intercept_

(array([  6686.2356258 ,   2284.47696322,    -49.07306427, -34774.59549927,
          2389.80189334,  32603.38021965]),
 9427.315722990774)

Получившееся уравнение:
$$\text{Суммарная капитализация} = 9427 + 6686*\sqrt[3]{\text{WTI}} + 2284*\sqrt[3]{\text{Brent}} - 49*\sqrt[3]{\text{Природный газ}} - 34774*\sqrt{\text{Usd-Rub}} + 2389*\text{Количество заболевших COVID-19, чел.} + 32603*\sqrt{\text{'Euro-Rub'}}$$


In [234]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_percentage_error

model = LinearRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
y_train_pred = model.predict(X_train)

print("Test MAPE = %.4f" % mean_absolute_percentage_error(y_test, y_pred))
print("Train MAPE = %.4f" % mean_absolute_percentage_error(y_train, y_train_pred))

Test MAPE = 0.1273
Train MAPE = 0.1215


уменьшаем MAPE

In [235]:
from sklearn.preprocessing import PolynomialFeatures

polinom_transform = PolynomialFeatures(2, include_bias=False)

X_train_poly = polinom_transform.fit_transform(X_train)
X_test_poly = polinom_transform.transform(X_test)

In [236]:
model = LinearRegression()

model.fit(X_train_poly, y_train)
y_pred = model.predict(X_test_poly)
y_train_pred = model.predict(X_train_poly)

print("Test MAPE = %.4f" % mean_absolute_percentage_error(y_test, y_pred))
print("Train MAPE = %.4f" % mean_absolute_percentage_error(y_train, y_train_pred))

Test MAPE = 0.0796
Train MAPE = 0.0661


In [237]:
mse = mean_squared_error(y_test, y_pred)
print("MSE:", mse)

rmse = mean_squared_error(y_test, y_pred, squared=False)
print("RMSE:", rmse)

r2 = r2_score(y_test, y_pred)
print("R-squared:", r2)

mape = mean_absolute_percentage_error(y_test, y_pred)
print("MAPE:", mape)

MSE: 4146004.7680611718
RMSE: 2036.1740515145486
R-squared: 0.6949288757884229
MAPE: 0.07962738154614644


Получаем, что при видоизмененных параметрах у нас получились следующие результаты: 
1. MSE упал 17375 ед
2. RMSE упал на 4 ед
3. R^2 возрос на 0.00135    
4. MAPE возросло на 0.002

Улучшилась модель по-сравнению с моделью №2 - при нормализации данных. Вследствие чего можно понять, что необходимые параметры (какие корни откуда изымаются) были подобраны успешно!

Наша итоговая модель линейной регрессии выглядит следующим образом:
$$\text{Суммарная капитализация} = 9427 + 6686\sqrt[3]{\text{WTI}} + 2284\sqrt[3]{\text{Brent}} - 49\sqrt[3]{\text{Природный газ}} - 34774\sqrt{\text{Usd-Rub}} + 2389\text{Количество заболевших COVID-19, чел.} + 32603\sqrt{\text{'Euro-Rub'}}$$

На основе этих данных можно сделать выводы о точности модели. Значение MSE и RMSE, хотя и высокие, но все же достаточно близки к фактическим значениям, что говорит о том, что модель довольно точна в прогнозировании. Коэффициент детерминации R-squared, близкий к 1, означает, что модель сравнительно хорошо описывает данные. Однако значение MAPE в 7,96% свидетельствует о том, что модель ошибается в среднем на 7,96%.